In [2]:
import math 
import string 
import re

import json 

import pandas as pd 
import sqlite3

ModuleNotFoundError: No module named 'pandas'

In [9]:
filepath = '/home/ramprasad.sa/factual_annotation_llm_summaries/generated_summaries/xsum_summaries_gpt-3.5-turbo-0301.csv'

In [10]:
df = pd.read_csv(filepath)
df

,Unnamed: 0,Unnamed: 0.1,article,dataset,Faithful Summary,Generic Summary,summ_id
0,0,0,"With three defeats in his previous six bouts, ...",xsum,Scottish boxer Ricky Burns has become the firs...,Scottish boxer Ricky Burns has won his third w...,bcb74d62d4ea11edaf5f1b63c3919b11
1,1,1,The 25-year-old man reportedly calls himself H...,xsum,A man believed to be in his mid-20s has been s...,A 25-year-old man named Harald Hitler has been...,bcb74f88d4ea11edaf5f1b63c3919b11
2,2,2,Kamal C Chavara was detained by the police in ...,xsum,Indian writer-activist Kamal C Chavara has bee...,An Indian writer-activist has been detained an...,bcb75046d4ea11edaf5f1b63c3919b11
3,3,3,"Mr Roache, who is secretary of the union's Yor...",xsum,Tim Roache has been elected as the new general...,Tim Roache has been elected as the new general...,bcb750d2d4ea11edaf5f1b63c3919b11
4,4,4,"The group, reported to be three men, two women...",xsum,"A British family, including four children aged...","A group of nine Britons, including four childr...",bcb751f4d4ea11edaf5f1b63c3919b11
...,...,...,...,...,...,...,...
95,95,95,Inspectors criticised teaching and leadership ...,xsum,Four schools run by the Telford Co-operative M...,The Telford Co-operative Multi-Academy Trust h...,bcb77c56d4ea11edaf5f1b63c3919b11
96,96,96,"Speaking to the Atlantic magazine, he said the...",xsum,US President Barack Obama has criticised Prime...,US President Barack Obama has criticised Briti...,bcb77cc4d4ea11edaf5f1b63c3919b11
97,97,97,The Labour authority voted in favour of changi...,xsum,Durham County Council has voted to cut the pay...,"Durham County Council has voted to reduce 2,70...",bcb77d32d4ea11edaf5f1b63c3919b11
98,98,98,She was hurt when the MR-01 race car she was d...,xsum,"Maria de Villota, a 32-year-old female racing ...","Maria de Villota, a Formula 1 test driver for ...",bcb77da0d4ea11edaf5f1b63c3919b11


In [11]:
import uuid
  
uuid.uuid1().hex

'8dc1c9ded4ee11ed9648b7178ea1f8cf'

In [12]:
# unique_ids = [uuid.uuid1().hex for i in range(0, len(df))]
# df['summ_id'] = unique_ids
# df.to_csv(filepath)

In [18]:
db_path = "../data/summaries_test.db"  
def connect_to_db():
    conn = sqlite3.connect(db_path)
    c = conn.cursor()
    return conn, c 
conn, c = connect_to_db()

In [16]:
for idx, row in df.iterrows():
    summ_uuid = row['summ_id']
    summ_id = 'xsum_generic'
    system_id = 'gpt3.5_turbo'
    article = row['article']
    generic_summary = row['Generic Summary']
    faithful_summary = row['Faithful Summary']
    c.execute("""INSERT INTO generated_summaries (summary_uuid, summ_id, system_id, summary, article) VALUES (?, ?, ?, ?, ?)""",
                                                        (summ_uuid, 
                                                        summ_id,
                                                        system_id,
                                                        generic_summary,
                                                        article))
print('Added %d generated summaries'%(idx))
conn.commit()
# conn.close()
    

Added 99 generated summaries


In [27]:
next_uuid = c.execute('''SELECT uuid FROM generated_summaries WHERE NOT EXISTS (
                    SELECT * FROM label WHERE generated_summaries.summary_uuid = label.summary_uuid) 
                      ORDER BY uuid, RANDOM() LIMIT 1;''').fetchone()

In [28]:
next_uuid

(1,)

In [35]:
summary_uuid = c.execute("""SELECT summary_uuid FROM generated_summaries where uuid='{}';""".format(next_uuid[0])).fetchone()
    
    

In [36]:
summary_uuid

('bcb74d62d4ea11edaf5f1b63c3919b11',)

In [43]:
article, summary = c.execute("""SELECT article, summary FROM generated_summaries where summary_uuid='{}';""".format(summary_uuid[0])).fetchone()


In [44]:
article

'With three defeats in his previous six bouts, the Scottish boxer\'s career seemed to be on the wane. But his power, timing and focus returned in a supremely confident display at Glasgow\'s Hydro on Saturday night as he clinched the vacant WBA World Super-Lightweight title - and entered the record books as the first Scot to win world titles at three weights. Italy\'s Michele Di Rocco, at 34 a year older than Burns, hadn\'t lost a fight since September 2007. He arrived with a record of 40 wins and a draw from 42 bouts. He left a lesser figure. The referee deemed the man from Assisi to be in no state to continue in the eighth round, the result of a welter of blows to his head and body and made all the more unbearable by the hostile acoustic energy generated by 8,000 Scottish fans. Surely, for Burns, this was time to shout about his record, the ideal moment to counter all those who had written him off and questioned the wisdom of him leaving trainer Billy Nelson two years ago to work with